Windows Server Information 

Gather Windows Log information 

Running Gather Log Events.ps1 as an administrator in PowerShell ISE (if applicable; if not, run in PowerShell as administrator) will collect error messages from the last 30 days found in the Application, Security and/or System Event Logs. 

The script will export the data to a CSV file (“export.csv”), which can be found in the C:\temp folder on the server in which the script is run. 

Power Options – High Performance should always be selected.   

Check to make sure High Performance is the selected Power Option, and that GPO has not made changes. 

Check for possible new anti-virus software on the server. If found, recommend excluding SQL files from the anti-virus scan. 

http://support.microsoft.com/kb/309422 

SQL Server Information
Server Properties
1.	Evaluate current SQL Server version. Report if there is a recent SP (<=2016) or CU (>=2017)
a.	Version reference: https://sqlcollaborative.github.io/builds
b.	There is a specific issue with TDE on early versions of 2016 that is critical. Need to make sure we are patched above CU7 2016 or CU4 2016 SP1 if using TDE.
i.	https://support.microsoft.com/en-us/help/4019893/fix-restore-fails-when-you-do-backup-by-using-compression-and-checksum
2.	Review SQL Server Default locations in Server Properties. The default locations must exist and could cause a service pack or CU install to fail if they do not.
3.	SQL Server Log 
a.	Review the SQL Server Error Log using Error Log.sql. Document any new/suspicious activity.
4.	System Configuration Values 
a.	Configuration Changes History.sql checks for system configuration changes.


In [ ]:
--error_log.sql
--#TODO: review string filters at bottom.
--Can execute in a multiserver query
--Execute in Grid mode

PRINT ('Error Log Output Window')
use tempdb
go
select SYSDATETIMEOFFSET()
declare @oldestdate as date, @now as datetime2(0)
select @oldestdate = dateadd(month,-3, sysdatetime()), @now = sysdatetime() --Filter the time frame of the logs.

select 'Getting errors since ' + cast(@oldestdate as varchar(30))

--Get list of logs associated with the SQL Server (by default is 7, probably need more!) 
CREATE TABLE #SQLErrorLogList (
    LogNumber INT NOT NULL,
    LogEndDate datetime2(0) NOT NULL,
    LogSize_b BIGINT NOT NULL);
CREATE NONCLUSTERED INDEX IDX_CL_ell on #SQLErrorLogList (LogNumber) INCLUDE (LogEndDate);

INSERT INTO #SQLErrorLogList
EXEC sys.sp_enumerrorlogs;

--error messages in current log
create table #readerrorlog
( LogDate datetime not null
, LogProcessInfo varchar(255) not null 
, [LogMessageText] varchar(1500) not null 
)

CREATE CLUSTERED INDEX IDX_CL_rel on #readerrorlog (LogDate);

declare @lognumber int = 0, @endoflogfiles bit = 0, @maxlognumber int = 0;

select @maxlognumber =   MAX(LogNumber) from #SQLErrorLogList
WHILE (
		((Select LogEndDate from #SQLErrorLogList where @lognumber = LogNumber) > @oldestdate)
		and @lognumber <= @maxlognumber
		) 
BEGIN

	INSERT INTO #readerrorlog 
	EXEC master.dbo.xp_readerrorlog  
	  @lognumber		--current log file
	, 1					--SQL Error Log
	, N''				--search string 1, must be unicode. Leave empty on purpose, as we do filtering later on.
	, N''				--search string 2, must be unicode. Leave empty on purpose, as we do filtering later on.
	, @oldestdate, @now --time filter. Should be @oldestdate < @now
	, N'desc'			--sort
			
	--print 'including lognumber ' + str(@lognumber)

	set @lognumber = @lognumber + 1	
END
GO

CREATE NONCLUSTERED INDEX IDX_NC_rel on #readerrorlog (Logdate desc, [LogMessageText]) INCLUDE( LogProcessInfo)

GO
--order of servers in a multiserver query is not determinant

--Raw error list
select * from #readerrorlog 
where  1=1
and (	
	LogMessageText like '%error%'
or	LogMessageText like '%failure%'
or	LogMessageText like '%failed%'
or	LogMessageText like '%corrupt%'
)
and LogMessageText not like '%without errors%'
and LogMessageText not like '%returned no errors%'
and LogMessageText not like 'Registry startup parameters:%'
and LogMessageText not like '%informational%'
and LogMessageText not like '%found 0 errors%'
order by LogDate desc;

--Aggregate error counts
select LogMessageText, LogProcessInfo, ErrorCount = count(LogDate), MostRecentOccurrence = max(LogDate) 
from #readerrorlog 
where  1=1
and (	
	LogMessageText like '%error%'
or	LogMessageText like '%failure%'
or	LogMessageText like '%failed%'
or	LogMessageText like '%corrupt%'
)
and LogMessageText not like '%without errors%'
and LogMessageText not like '%returned no errors%'
and LogMessageText not like 'Registry startup parameters:%'
and LogMessageText not like '%informational%'
and LogMessageText not like '%found 0 errors%'
group by LogMessageText, LogProcessInfo
order by count(LogDate) desc, max(LogDate) desc;

SELECT Reboots = LogDate FROM #readerrorlog WHERE LogMessageText like 'Registry startup parameters:%'
ORDER BY LogDate desc;
GO

drop table #readerrorlog
drop table #SQLErrorLogList

In [ ]:
--Configuration Changes History.sql
--Based on the configuration changes history report in SSMS
PRINT('Configuration Change History Output Window')
exec sp_executesql @stmt=N'begin try
declare @enable int;
select @enable = convert(int,value_in_use) from sys.configurations where name = ''default trace enabled''
if @enable = 1 --default trace is enabled
begin
        declare @d1 datetime;
        declare @diff int;  
        declare @curr_tracefilename varchar(500); 
        declare @base_tracefilename varchar(500); 
        declare @indx int ;
        declare @temp_trace table (
                textdata nvarchar(MAX) collate database_default 
        ,       login_name sysname collate database_default
        ,       start_time datetime
        ,       event_class int
        );
        
        select @curr_tracefilename = path from sys.traces where is_default = 1 ; 
        
        set @curr_tracefilename = reverse(@curr_tracefilename)
        select @indx  = PATINDEX(''%\%'', @curr_tracefilename) 
        set @curr_tracefilename = reverse(@curr_tracefilename)
        set @base_tracefilename = LEFT( @curr_tracefilename,len(@curr_tracefilename) - @indx) + ''\log.trc'';
        
        insert into @temp_trace
        select TextData
        ,       LoginName
        ,       StartTime
        ,       EventClass 
        from ::fn_trace_gettable( @base_tracefilename, default ) 
        where ((EventClass = 22 and Error = 15457) or (EventClass = 116 and TextData like ''%TRACEO%(%''))
        
        select @d1 = min(start_time) from @temp_trace
        
        --set @diff= datediff(hh,@d1,getdate())
        --set @diff=@diff/24; 

        select --(row_number() over (order by start_time desc))%2 as l1
                @d1 as TraceStartDate
        ,       start_time as EventDate
		,       case event_class 
                        when 116 then ''Trace Flag '' + substring(textdata,patindex(''%(%'',textdata),len(textdata) - patindex(''%(%'',textdata) + 1) 
                        when 22 then substring(textdata,58,patindex(''%changed from%'',textdata)-60) 
                end as config_option
        ,       login_name
        ,       case event_class 
                        when 116 then ''--''
                        when 22 then substring(substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))
                                                                ,patindex(''%changed from%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata)))+13
                                                                ,patindex(''%to%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))) - patindex(''%from%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))) - 6) 
                end as old_value
        ,       case event_class 
                        when 116 then substring(textdata,patindex(''%TRACE%'',textdata)+5,patindex(''%(%'',textdata) - patindex(''%TRACE%'',textdata)-5)
                        when 22 then substring(substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))
                                                                ,patindex(''%to%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata)))+3
                                                                , patindex(''%. Run%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))) - patindex(''%to%'',substring(textdata,patindex(''%changed from%'',textdata),len(textdata) - patindex(''%changed from%'',textdata))) - 3) 
                end as new_value
        from @temp_trace 
        order by start_time desc
end else 
begin 
        select top 0  1  as l1, 1 as difference,1 as date , 1 as config_option,1 as start_time , 1 as login_name, 1 as old_value, 1 as new_value
end
end try 
begin catch
select  ERROR_NUMBER() as Error_Number
,       ERROR_SEVERITY() as date 
,       ERROR_STATE() as config_option
,       1 as start_time 
,       ERROR_MESSAGE() as login_name
,       1 as old_value, 1 as new_value
end catch',@params=N''

Performance
1.	Run Page Life Expectancy.sql.
a.	Does SQL have enough memory, based on PLE, Churn, and the Target vs Total?
2.	Review CPU Utilization.Sql script and look for any recent +90% periods.
3.	Are there any recent Memory Dumps?
a.	Running Find Memory Mini Dumps.sql will let you know if SQL is having issues or has had any  “mini dump crashes” in the recent past.



In [ ]:
--Page Life Expectancy.sql
PRINT('Page Life Expectancy')
select 
	p.InstanceName
,	c.Version 
,	'LogicalCPUCount'		= os.cpu_count
,	OS_Physical_Mem_MB = os.[Server Physical Mem (MB)] -- SQL2012+ only
,	Min_Server_Mem_MB = c.[Min_Server_Mem_MB]
,	Max_Server_Mem_MB = c.[Max_Server_Mem_MB] --2147483647 means unlimited, just like it shows in SSMS
,	p.PLE_s --300s is only an arbitrary rule for smaller memory servers (<16gb), for larger, it should be baselined and measured.
,	'Churn (MB/s)'			=	cast((p.Total_Server_Mem_GB)/1024./NULLIF(p.PLE_s,0) as decimal(19,2))
,	OS_Available_physical_mem_GB = (SELECT cast(available_physical_memory_kb / 1024. / 1024. as decimal(19,2)) from sys.dm_os_sys_memory) 
,	SQL_Physical_memory_in_use_GB = (SELECT cast(physical_memory_in_use_kb / 1024. / 1024. as decimal(19,2)) from sys.dm_os_process_memory)
,	p.Total_Server_Mem_GB --May be more or less than memory_in_use 
,	p.Target_Server_Mem_GB	
,	Target_vs_Total = CASE WHEN p.Total_Server_Mem_GB < p.Target_Server_Mem_GB	 
							THEN 'Target >= Total. SQL wants more memory than it has, or is building up to that point.'
							ELSE 'Total >= Target. SQL has enough memory to do what it wants.' END
,	si.LPIM -- Works on SQL 2016 SP1, 2012 SP4+
from(
select 
	InstanceName = @@SERVERNAME 
,	Target_Server_Mem_GB =	max(case counter_name when 'Target Server Memory (KB)' then convert(decimal(19,3), cntr_value/1024./1024.) end)
,	Total_Server_Mem_GB	=	max(case counter_name when  'Total Server Memory (KB)' then convert(decimal(19,3), cntr_value/1024./1024.) end) 
,	PLE_s	=	max(case counter_name when 'Page life expectancy'  then cntr_value end) 
--select * 
from sys.dm_os_performance_counters
--This only looks at one NUMA node. https://www.sqlskills.com/blogs/paul/page-life-expectancy-isnt-what-you-think/
)  as p
inner join (select 'InstanceName' = @@SERVERNAME, Version = @@VERSION, 
			Min_Server_Mem_MB  = max(case when name = 'min server memory (MB)' then convert(bigint, value_in_use) end) ,
			Max_Server_Mem_MB = max(case when name = 'max server memory (MB)' then convert(bigint, value_in_use) end) 
			from sys.configurations) as c on p.InstanceName = c.InstanceName
inner join (SELECT 'InstanceName' = @@SERVERNAME 
			, cpu_count , hyperthread_ratio AS 'HyperthreadRatio',
			cpu_count/hyperthread_ratio AS 'PhysicalCPUCount'
			, 'Server Physical Mem (MB)' = cast(physical_memory_kb/1024. as decimal(19,2))   -- SQL2012+ only
			FROM sys.dm_os_sys_info ) as os
on c.InstanceName=os.InstanceName


-- Works on SQL 2016 SP1, 2012 SP4+
cross apply (select LPIM = CASE sql_memory_model_Desc 
					WHEN  'Conventional' THEN 'Lock Pages in Memory privilege is not granted'
					WHEN 'LOCK_PAGES' THEN 'Lock Pages in Memory privilege is granted'
					WHEN 'LARGE_PAGES' THEN 'Lock Pages in Memory privilege is granted in Enterprise mode with Trace Flag 834 ON'
					END from sys.dm_os_sys_info 
				) as si

--adapted from http://www.datavail.com/category-blog/max-server-memory-300-second-rule/


In [ ]:
--CPU Utilization.sql
--This is simple use of the ring_buffer for historical CPU, goes back a little over 4 hours.
-- for more CPU and Memory, look at toolbox/sys_dm_os_ring_buffers.sql
PRINT('CPU Utilization Output Window')
select
	Avg_SystemIdle_Pct				=	AVG( record.value('(./Record/SchedulerMonitorEvent/SystemHealth/SystemIdle)[1]', 'int') )
,	Avg_SQLProcessUtilization_Pct	=	AVG( record.value('(./Record/SchedulerMonitorEvent/SystemHealth/ProcessUtilization)[1]', 'int') )
      from (
            select timestamp, convert(xml, record) as record
            from sys.dm_os_ring_buffers
            where ring_buffer_type = N'RING_BUFFER_SCHEDULER_MONITOR'
            and record like '%<SystemHealth>%') as x

declare @ts_now bigint
--select @ts_now = cpu_ticks / convert(float, cpu_ticks_in_ms) from sys.dm_os_sys_info
select @ts_now = cpu_ticks / (cpu_ticks/ms_ticks) from sys.dm_os_sys_info;
select	record_id
	,	EventTime				=  dateadd(ms, -1 * (@ts_now - [timestamp]), GetDate()) 
	,	SQLProcessUtilization
	,	SystemIdle
	,	OtherProcessUtilization	= 100 - SystemIdle - SQLProcessUtilization 
from (
      select
            record_id				=	record.value('(./Record/@id)[1]', 'int')
        ,	SystemIdle				=	record.value('(./Record/SchedulerMonitorEvent/SystemHealth/SystemIdle)[1]', 'int') 
        ,	SQLProcessUtilization	=	record.value('(./Record/SchedulerMonitorEvent/SystemHealth/ProcessUtilization)[1]', 'int') 
        ,	timestamp
      from (
            select timestamp, convert(xml, record) as record
            from sys.dm_os_ring_buffers
            where ring_buffer_type = N'RING_BUFFER_SCHEDULER_MONITOR'
            and record like '%<SystemHealth>%') as x
      ) as y
order by record_id desc


--http://sqlblog.com/blogs/ben_nevarez/archive/2009/07/26/getting-cpu-utilization-data-from-sql-server.aspx

In [ ]:
--Find Memory Mini Dumps.sql 
-- Get information on location, time and size of any memory dumps from SQL Server  
-- Only SQL 2008R2+
PRINT('Fine Memory Mini Dump Output Window')
SELECT [filename], creation_time, size_in_bytes/1048576.0 AS [Size (MB)]
FROM sys.dm_server_memory_dumps 
ORDER BY creation_time DESC OPTION (RECOMPILE);

SQL Agent Configuration
1.	Are basic alerts for severity 16 and higher setup and enabled to the correct operator, covering all databases?
a.	If not, recommend running Add Error Event Alerts.sql to enable alert notification. 
b.	Consider disabling the severity 20 error alert because of common nuisance emails.
2.	Are the following jobs enabled on the server and completing successfully? Check Job History on the following jobs to confirm all are succeeding: 
a.	Automated Index Maintenance
i.	Look for any type of similarly-named index maintenance job to ensure index maintenance is being completed routinely.
ii.	Actually check the log table to verify activity and no errors. This or similar:
select * from dbaadmin.dbo.indexmaintlog order by id desc 
b.	Volume Stats Insert
i.	Do a select on the table, this should get a few months of data back. Any big jumps or trends?
SELECT TOP (1000) *  FROM [DBAHOUND].[DBO].[VOLUMESTATS]  ORDER BY DATETIMEPERFORMED DESC
a.	Job Failure Notification
ii.	Can also be named “Add Job Failure Notification”
3.	Change the SQL Server Agent log history retention. Check how far back history goes for important jobs, such as backup jobs. Make sure to change the total row count from default 1000 rows to 10000 or larger, especially on servers that have replication jobs running or multiple large-output jobs, to prevent loss of historical Information. To store hourly transaction log backups rows for 3 months (recommended), you would also need to increase the max rows per job to at least 2160!

In [ ]:
SELECT TOP (1000) * FROM [DBALOGGING].[DBO].[VOLUMESTATS] ORDER BY DATETIMEPERFORMED DESC

Drives
1.	Record SQL file locations
a.	Run “select * from sys.master_files”
i.	Are there any SQL server files that have been added to the C: drive? If so, they should be moved.

In [ ]:
select * from sys.master_files

Databases
4.	Review Database Configurations: note new databases, record any changes to existing, etc.
a.	Database Settings Info.sql
i.	Included in this script is the sys.databases.log_reuse_wait value for each database.
1.	If 0 or 2, this is not an issue and is normal.
2.	If 4, this may indicate an uncommitted transaction is open and needs to be cleaned up. Check uncommitted transactions.sql.
3.	If 6 or 9, this requires some attention to the replication or Availability Groups, respectively.
4.	If other values, investigate accordingly. https://docs.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-databases-transact-sql?view=sql-server-2017
5.	Make sure autogrowth settings are NOT set to 1 MB on any files using Autogrow Change All 1 MB Growth Files.sql
a.	We recommend changing the autogrowth settings to a fixed rate between 128 MB and 512 MB depending on the size of the database.
6.	Record any autogrowth events from the past month using Autogrowth Events.sql.
7.	Use Space in files.sql to report any files short on space that are in danger of an autogrowth event.
a.	We typically report any files with less than 5% free space available, but if the server is checked less frequently than once per month, we want to report anything with less than 10% free space available and also want to manually grow that file to prevent autogrowth events until the next cycle of health checks.
b.	Are any log files full? This would indicate FULL recovery + no TLOG backups.
8.	For SharePoint environments, evaluate the correct Recovery model for databases. This cannot be reliably done by database name, so execute the following .ps1 script on a web or app server in the sharepoint farm, or the SQL server if it serves as a single-server farm.
a.	Sharepoint Databases.ps1
b.	https://technet.microsoft.com/en-us/library/cc678868(v=office.16).aspx
9.	VLF Counts
a.	Check the number of VLF’s in all database using VLFs Analysis.sql. Too many VLF’s can be bad for the performance of restore operations and slow recovery operations include the amount of time it takes for a database to come online after a SQL restart. Take a transaction log backup to empty the log file as much as possible, and then shrink the transaction to near-zero using a shrink operation.  Re-grow the log file back to an appropriate size in 8000MB increments. The script handles this.
10.	Ownership
a.	Individual employees should not own objects on SQL Server.  In the event their account is LOCKED or DISABLED it could cause problems. Look for this especially on newly created databases.
i.	Database Ownership.sql
ii.	Job Owners.sql
11.	Check tempdb data file size – they should all be the same size and have the same autogrowth setting. The initial TempDB data files sizes should be equal to the current TempDB data file sizes.
i.	Follow the recommendations in Tempdb Data Files.sql
ii.	Pre-grow tempdb data files large so that they never autogrow. If this is a constant issue that cannot be fixed with large initial TempDB sizes (this is rare), consider 1117 – Grow All Files in a FileGroup Equally in SQL versions prior to SQL 2016. 
12.	Age of Statistics –update indexes with outdated statistics (typically by business cycle =  monthly)
i.	Stats Out of Date whileloop.sql to review all databases.
13.	If we have replication, check the Replication Monitor in SSMS. Look for recent errors.
a.	Verify that “there are no replicated transactions available” for replication scenarios that are nightly snapshot only.
b.	Verify no error messages for other replication scenarios.
14.	For SQL 2016+, turn on Query Store on any performance-sensitive database. Use default settings. Set to read/write.
a.	select name, is_query_store_on from sys.databases 
15.	Use last known DBCC CHECKDB.sql to verify that a CHECKDB has been run on all databases in at least the last month. If not, we need to examine maintenance plans.

In [ ]:
--Database settings infor.sql
PRINT('Database Settings Output Window')
IF OBJECT_ID('tempdb..#DBSettings') IS NOT NULL
    BEGIN
	   DROP TABLE #DBSettings;
    END;
GO
select 
	name
,	[compatibility_level]	
,	[dbstate] = case when state_desc = 'online' and is_read_only = 1 then state_desc + ' ' +'(Read-Only)' else state_desc end 		
,	recovery_model_desc
,	page_verify_option_desc
,	user_access_desc				--should be MULTI_USER
,	is_auto_close_on				--should be 0
,	is_auto_shrink_on				--should be 0
,	is_auto_create_stats_on			--should be 1 except for some SharePoint db's
,	is_auto_update_stats_on			--should be 1 except for some SharePoint db's
,	is_auto_update_stats_async_on	--should be 1 except for some SharePoint db's
,	log_reuse_wait
,	log_reuse_wait_desc
,	target_recovery_time_in_seconds
,	ProductMajorVersion				= SERVERPROPERTY('ProductMajorVersion')
,	is_trustworthy_on
into #DBSettings
from sys.databases;

--Compatibility Level Check
WITH cteDB (Database_Name, [compatibility_level], State, Up_To_Date)
AS (
SELECT 
 	Database_Name			= name
,	[Compatibility Level]	= [compatibility_level] --should be latest (130 = SQL2016, 120 = SQL2014, 110 = SQL2012, 100 = SQL2008, 90 = SQL2005)
,	[State]					= dbstate		
,	Up_To_Date				= cast(ProductMajorVersion as char(2)) + '0'
from #DBSettings
)
select
	cteDB.*
,	[SQL Server Version]	= SERVERPROPERTY('ProductVersion')
,	[Alter]					= CASE WHEN Up_To_Date is not null THEN 'ALTER DATABASE [' + Database_Name +'] SET COMPATIBILITY_LEVEL = ' + LEFT(convert(varchar(15), SERVERPROPERTY('ProductVersion')),2) + '0;' ELSE NULL END
,	[Revert]				= CASE WHEN Up_To_Date is not null THEN 'ALTER DATABASE [' + Database_Name +'] SET COMPATIBILITY_LEVEL = ' + convert(char(3), [compatibility_level]) + ';' ELSE NULL END
from cteDB
WHERE Up_To_Date <> [compatibility_level]
and state <> 'OFFLINE'
order by [Database_Name];


--Databases where page verify option is not CHECKSUM
--Changing this setting does not instantly put a checksum on every page. Need to do an index REBUILD of all objets to get CHECKSUMS in place, or, it'll happen slowly over time as data is written.
select
 	[Database Name]			= name
,	[Page Verify Option]	= page_verify_option_desc
,	[Message]				= 'Page Verify Option MUST be CHECKSUM!'
,	[Alter]					= 'ALTER DATABASE [' + name +'] SET PAGE_VERIFY CHECKSUM WITH NO_WAIT; --Need to rebuild indexes on all objects in DB to take effect '
,	[Revert]				= 'ALTER DATABASE [' + name +'] SET PAGE_VERIFY ' + page_verify_option_desc COLLATE DATABASE_DEFAULT + ' WITH NO_WAIT;'
,	[State]					= dbstate		
from #DBSettings
where page_verify_option_desc <> 'CHECKSUM'
ORDER BY name;

--Databases where auto-close and/or auto-shrink is enabled. 
--Strongly recommend NEVER enabling either of these two settings.
select 
 	[Database Name]			= name
,	[Is Auto Close On]		= is_auto_close_on		--should be 0
,	[Is Auto Shrink On]		= is_auto_shrink_on		--should be 0
,	[Alter]					= CASE
									WHEN is_auto_close_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CLOSE OFF WITH NO_WAIT;'
									WHEN is_auto_shrink_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_SHRINK OFF WITH NO_WAIT;'
									WHEN is_auto_close_on = 1 AND is_auto_shrink_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CLOSE OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_SHRINK OFF WITH NO_WAIT;'
							  ELSE 'N/A'
							  END
,	[Revert]				= CASE
									WHEN is_auto_close_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CLOSE ON WITH NO_WAIT;'
									WHEN is_auto_shrink_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_SHRINK ON WITH NO_WAIT;'
									WHEN is_auto_close_on = 1 AND is_auto_shrink_on = 1 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CLOSE ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_SHRINK ON WITH NO_WAIT;'
							  ELSE 'N/A'
							  END
,	[State]					= dbstate	
from #DBSettings
where is_auto_close_on = 1		
   OR is_auto_shrink_on	= 1	
ORDER BY name;

--Databases where auto create and/or auto update stats is disabled
--Recommend enabling these settings.
select 
	[Database Name]					= name
,	[Is Auto Create Stats On]		= is_auto_create_stats_on		--should be 1 except for some SharePoint db's
,	[Is Auto Update Stats On]		= is_auto_update_stats_on		--should be 1 except for some SharePoint db's
,	[Is Auto Update Stats Async On]	= is_auto_update_stats_async_on	--should be 1 except for some SharePoint db's
,	ProductVersion = SERVERPROPERTY('ProductVersion')
,	[Alter]							= CASE
											WHEN is_auto_create_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS ON WITH NO_WAIT;'
											WHEN is_auto_update_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS ON WITH NO_WAIT;'
											WHEN is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC ON WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS ON WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC ON WITH NO_WAIT;'
											WHEN is_auto_update_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC ON WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS ON WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC ON WITH NO_WAIT;'
									  ELSE 'N/A'
									  END
,	[Revert]						= CASE
											WHEN is_auto_create_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS OFF WITH NO_WAIT;'
											WHEN is_auto_update_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS OFF WITH NO_WAIT;'
											WHEN is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC OFF WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS OFF WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC OFF WITH NO_WAIT;'
											WHEN is_auto_update_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC OFF WITH NO_WAIT;'
											WHEN is_auto_create_stats_on = 0 AND is_auto_update_stats_on = 0 AND is_auto_update_stats_async_on = 0 THEN 'ALTER DATABASE [' + name + '] SET AUTO_CREATE_STATISTICS OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS OFF WITH NO_WAIT; ALTER DATABASE [' + name + '] SET AUTO_UPDATE_STATISTICS_ASYNC OFF WITH NO_WAIT;'
									  ELSE 'N/A'
									  END
,	[State]							= dbstate	
from #DBSettings
where is_auto_create_stats_on = 0
   OR is_auto_update_stats_on = 0
   OR (is_auto_update_stats_async_on = 0 and ProductMajorVersion >= 12)
ORDER BY name;

--Databases log reuse wait and description
--Expected types: NOTHING, CHECKPOINT, LOG_BACKUP, ACTIVE_BACKUP_OR_RESTORE, DATABASE_SNAPSHOT_CREATION, AVAILABILITY_REPLICA, OLDEST_PAGE, XTP_CHECKPOINT
--Potentially problematic if long-lasting, research: DATABASE_MIRRORING, REPLICATION, ACTIVE_TRANSACTION, LOG_SCAN, OTHER_TRANSIENT 
select 
	[Database Name]		= name
,	[Log Reuse Wait]	= log_reuse_wait
,	[Description]		= log_reuse_wait_desc
,	[State]				= dbstate		
,	[Recovery Model]	= recovery_model_desc
from #DBSettings
where log_reuse_wait_desc not in ('NOTHING', 'CHECKPOINT', 'LOG_BACKUP', 'ACTIVE_BACKUP_OR_RESTORE', 'DATABASE_SNAPSHOT_CREATION', 'AVAILABILITY_REPLICA', 'OLDEST_PAGE', 'XTP_CHECKPOINT')
ORDER BY name;

--Databases where target recovery time in seconds is < 60 (only applies to 2012+), and recommended in 2016+
--Make sure latest patches are applied first.
select 
	[Database Name]			= name
,	[Target Recovery Time]	= target_recovery_time_in_seconds
,	[Alter]					= 'ALTER DATABASE [' + name + '] SET TARGET_RECOVERY_TIME = 60 SECONDS WITH NO_WAIT'
,	[Revert]				= 'ALTER DATABASE [' + name + '] SET TARGET_RECOVERY_TIME = ' + CAST(target_recovery_time_in_seconds AS VARCHAR(3)) + ' SECONDS WITH NO_WAIT'
,	[State]					= dbstate		
,	ProductMajorVersion
from #DBSettings 
where target_recovery_time_in_seconds = 0
and cast(ProductMajorVersion as int) >= 13
and [name] <> 'master'
ORDER BY name;


--Databases should only have the Trustworthy setting enabled if necessary. The msdb system database is Trustworthy by default. 
select 
	[Database Name]			= name
,	is_trustworthy_on
from #DBSettings
where is_trustworthy_on = 1
and name <> 'msdb'
ORDER BY name;

In [ ]:
--uncommitted transactions.sq
--Returns information on uncommitted transactions
PRINT('Uncommitted Transactions Output Window')
select * from sys.dm_tran_active_transactions tat 
inner join sys.dm_tran_session_transactions tst  on tat.transaction_id = tst.transaction_id

In [ ]:
--Autogrow Change All 1 MB Growth Files.sql
--Run in Results to Text mode
PRINT('Databses with Default Autogrowth Settings in Use')
USE [master]
GO
select 
Alter_Autogrowth_Rates = case when mf.type_desc = 'ROWS' 
	then 'ALTER DATABASE ['+d.name+'] MODIFY FILE ( NAME = N'''+ mf.name+ ''', FILEGROWTH = 256MB );
GO' 
	else 'ALTER DATABASE ['+d.name+'] MODIFY FILE ( NAME = N'''+ mf.name+ ''', FILEGROWTH = 256MB );
GO' 
	end
, mf.*
FROM sys.databases d
inner join sys.master_files mf
on d.database_id = mf.database_id
where (d.state_desc = 'ONLINE')
and (d.is_read_only = 0)
and ((mf.is_percent_growth = 0 and growth = 128) or (mf.is_percent_growth = 1))
/*
ALTER DATABASE [model] MODIFY FILE ( NAME = N'modeldev', FILEGROWTH = 512000KB )
*/


In [ ]:
--Autogrowth Events.sql

--The trace automatically finds _n files, strip off the _nnn. For example, will read all data from log_14.trc, log_15.trc, log_16.trc, log_17.trc, etc. 
--Default trace files are limited to 20mb, and there are at most five of them, so we have 100mb of history. Depends on activity to determine how far back that goes.
PRINT('Autogrowth Events Output Window')

	SELECT 
		DBName				=	g.DatabaseName
	,	DBFileName			=	mf.physical_name
	,	FileType			=	CASE mf.type WHEN 0 THEN 'Row' WHEN 1 THEN 'Log' WHEN 2 THEN 'FILESTREAM' WHEN 4 THEN 'Full-text' END
	,	EventName			=	te.name
	,	EventGrowthMB		=	convert(decimal(19,2),g.IntegerData*8/1024.) -- Number of 8-kilobyte (KB) pages by which the file increased.
	,	EventTime			=	g.StartTime
	,	EventDurationSec	=	convert(decimal(19,2),g.Duration/1000./1000.) -- Length of time (in milliseconds) necessary to extend the file.
	,	CurrentAutoGrowthSet=	CASE
									WHEN mf.is_percent_growth = 1
									THEN CONVERT(char(2), mf.growth) + '%' 
									ELSE CONVERT(varchar(30), convert(decimal(19,2), mf.growth*8./1024.)) + 'MB'
								END
	,	CurrentFileSizeMB	=	convert(decimal(19,2),mf.size*	8./1024.)
	,	d.Recovery_model_Desc
	--,	@tracepath	
	--,	MaxFileSizeMB		=	CASE WHEN mf.max_size = -1 THEN 'Unlimited' ELSE convert(varchar(30), convert(decimal(19,2),mf.max_size*8./1024.)) END
	--select count(1)
	FROM fn_trace_gettable((select substring((select path from sys.traces where is_default =1), 0, charindex('\log_', (select path from sys.traces where is_default =1),0)+4)	+ '.trc'), default) g
	cross apply sys.trace_events te 
	inner join sys.master_files mf
	on mf.database_id = g.DatabaseID
	and g.FileName = mf.name
	inner join sys.databases d
	on d.database_id = g.DatabaseID
	WHERE g.eventclass = te.trace_event_id
	and		te.name in ('Data File Auto Grow','Log File Auto Grow')
	and		g.StartTime > dateadd(d, -7, sysdatetime()) 
	--GROUP BY StartTime,Databaseid, Filename, IntegerData, Duration
	order by StartTime desc;

	SELECT servicename, status_desc, last_startup_time FROM sys.dm_server_services;
GO

In [ ]:
--Space in files.sql
--Observe space in data and log files
--Pregenerated scripts to shrink and/or grow files. Do not shrink unless an unusual/emergency situation has created an overgrown log file. 
--See also "vlfs analysis.sql"
PRINT('Space in Files Output Window')

DECLARE @TempTable TABLE
( DatabaseName varchar(128)
,recovery_model_desc varchar(50)
,DatabaseFileName varchar(500)
,FileLocation varchar(500)
,FileId int
,type_desc varchar(50)
,FileSizeMB decimal(19,2)
,SpaceUsedMB decimal(19,2)
,AvailableMB decimal(19,2)
,FreePercent decimal(19,2)
,shrinkTSQL nvarchar(4000)
,growTSQL nvarchar(4000)
)

--Optional filter for small/unused databases at bottom

INSERT INTO @TempTable
exec sp_MSforeachdb  'use [?]; 
select *
, shrinkTSQL	=	''USE [?];
DBCC SHRINKFILE (N''''''+ DatabaseFileName_______ COLLATE SQL_Latin1_General_CP1_CI_AS +'''''' , 0, TRUNCATEONLY)''
, growTSQL = ''ALTER DATABASE [''+DatabaseName_____________ COLLATE SQL_Latin1_General_CP1_CI_AS+''] 
MODIFY FILE ( NAME = N''''''+DatabaseFileName_______ COLLATE SQL_Latin1_General_CP1_CI_AS +''''''
, '' + CASE WHEN FileSizeMB < 100 THEN ''SIZE = ''+STR(FileSizeMB+64)
			WHEN FileSizeMB < 1000 THEN ''SIZE = ''+STR(FileSizeMB+256)
			WHEN FileSizeMB < 10000 THEN ''SIZE = ''+STR(FileSizeMB+1024)
			WHEN FileSizeMB < 40000 THEN ''SIZE = ''+STR(FileSizeMB+4092)
			ELSE ''SIZE = ''+STR(FileSizeMB+(FileSizeMB*.05)) END +''MB )''
FROM (
SELECT 
  ''DatabaseName_____________'' = d.name
, Recovery			= d.recovery_model_desc
, ''DatabaseFileName_______'' = df.name
, Location			= df.physical_name
, File_ID			= df.File_ID
, df.type_desc
, FileSizeMB		= CAST(size/128.0 as Decimal(9,2))
, SpaceUsedMB		= CAST(CAST(FILEPROPERTY(df.name, ''SpaceUsed'') AS int)/128.0 as Decimal(9,2))
, AvailableMB		= CAST(size/128.0 - CAST(FILEPROPERTY(df.name, ''SpaceUsed'') AS int)/128.0 as Decimal(9,2))
, FreePercent		= CAST((((size/128.0) - (CAST(FILEPROPERTY(df.name, ''SpaceUsed'') AS int)/128.0)) / (size/128.0) ) * 100. as Decimal(9,2))

 FROM sys.database_files df
 CROSS APPLY sys.databases d
 WHERE d.database_id = DB_ID() 
 AND d.is_read_only = 0
 AND df.size > 0) x;
'

SELECT
    *
FROM @TempTable
--where [FreePercent] < 5  and FileSizeMB > 6 --Optional filter for small/unused databases 

ORDER BY FreePercent asc, DatabaseName, FileId


In [ ]:
--VLFs Analysis.sql

--TODO: Rewrite this for SQL 2016 SP2+ and SQL 2017 using sys.dm_db_log_stats instead of DBCC LOGINFO

--If no TSQL scripts generated in the Messages tab, then no log files found with need for VLF maint.

--shows the number of VLF's. CreateLSN=0 for the original created files.
--mf.size /1024, *8 to get MB 
--LogInfo filesize is in bytes, so /1024./1024. to get MB
--Ideally 1 VLF per 500MB. 
--IF log >8 GB, Recreate log in 8000 MB increments.
--A single VLF larger than 4GB could be a problem with backup compression - https://techcommunity.microsoft.com/t5/SQL-Server/Backup-Compression-for-TDE-enabled-Databases-Important-fixes-in/ba-p/385593?advanced=false&collapse_discussion=true&q=MAXTRANSFERSIZE&search_type=thread

--Shrink/regrow step only works for databases with one log file. Why do you have more than one log file anyway? Stop. Think. Ask yourself.

PRINT('VLF Analysis Output Window')
BEGIN TRY
IF EXISTS (select * from tempdb.sys.objects where name like '#Log%')
DROP TABLE #Log
END TRY
BEGIN CATCH
END CATCH

SET NOCOUNT ON

Create Table #Log(
	RecoveryUnitId bigint  null,--SQL 2012 and above only, comment out for <=SQL 2008
    FileID      int not null
  , FileSize_b    bigint not null
  , StartOff	bigint not null
  , FSeqNo      bigint not null
  , [Status]    int not null
  , Parity      bigint not null
  , CreateLSN   decimal(30,0) not null
);  
Exec sp_MSforeachdb N'Use [?]; 
Insert Into #Log  
Exec sp_executesql N''DBCC LogInfo([?]) with no_infomsgs''; 
declare @Co bigint, @Avg_MB decimal(19,2), @LCnt int, @Log_MB decimal(19,2) , @T nvarchar(4000), @Max_MB bigint
select @Log_MB =sum(convert(bigint, mf.size))*8/1024 FROM sys.master_files mf where type=1 and state=0 and db_id()=mf.database_id
select @Co=Count_big(StartOff) ,	@Avg_MB=@Log_MB / Count_big(StartOff), @Max_MB=MAX(FileSize_b/1048576.) from #Log
if (((@Avg_MB <= 64 OR @Avg_MB > 4000) AND @Log_MB > 1024) AND (@Log_MB<8000)) AND EXISTS 
(select 1 FROM sys.databases WHERE is_read_only=0 and state=0 and db_id()=database_id)
BEGIN
		select DBName= db_name(), Co=@Co, Size_MB=@Log_MB, Avg_MB=@Avg_MB
SELECT @T= ''USE [''+d.name+''];
CHECKPOINT
GO
DBCC SHRINKFILE (N''''''+mf.name+'''''' , 0, TRUNCATEONLY)
GO
USE [master]
--Original Size ''+convert(varchar(1000), @Log_MB) +'' MB
ALTER DATABASE [''+d.name+''] MODIFY FILE ( NAME=N''''''+mf.name+'''''', SIZE=''+convert(varchar(30), mf.size*8/1024)+''MB );
GO
''
FROM sys.databases d inner join sys.master_files mf on d.database_id=mf.database_id where type_desc=''log'' and db_name()=d.name
IF @T IS NOT NULL BEGIN
	set @T=@T+''
''
	IF @Co>(@Log_MB/100) and @Co>50
	SELECT DB_NAME()+'' log file too many VLFs.''
	IF (@Avg_MB<64 AND @Log_MB>1024) and @Co>50
	SELECT DB_NAME()+'' log file VLFs too small.''
	IF (@Max_MB >= 4096) 
	SELECT DB_NAME()+'' single log file VLF >4000MB''
	
	print  @T
END	
END
Truncate Table #Log;'


--Had to split this out because of sp_MSforeachdb char limits.
Exec sp_MSforeachdb N'Use [?]
Insert Into #Log  
Exec sp_executesql N''DBCC LogInfo([?]) with no_infomsgs''; 
DECLARE @Co bigint, @Avg_MB decimal(19,2), @LCnt int, @Log_MB decimal(19,2) , @Log_curr bigint, @T nvarchar(4000), @LNeed int, @Ac bigint, @Max_MB bigint
SELECT @Log_MB=sum(convert(bigint, mf.size))*8./1024. FROM sys.master_files mf where type=1 and state=0 and db_id()=mf.database_id
SELECT @Co=Count_big(StartOff),@Avg_MB=@Log_MB/Count_big(StartOff), @Max_MB=MAX(FileSize_b/1048576.) from #Log
IF (@Avg_MB>1024 OR (@Avg_MB<64 AND @Log_MB > 1024)) AND (@Log_MB>8000) AND EXISTS (select 1 FROM sys.databases WHERE is_read_only=0 and state=0 and db_id()=database_id)
BEGIN
SELECT @LCnt=1, @Ac=0
SELECT top 1 @T=''USE [''+d.name+'']
CHECKPOINT
GO
DBCC SHRINKFILE (N''''''+mf.name+'''''' , 0, TRUNCATEONLY)
GO
USE master
GO
--Orig ''+convert(varchar(1000),@Log_MB) +'' MB
''
FROM sys.databases d join sys.master_files mf on d.database_id=mf.database_id where type_desc=''log'' and db_name()=d.name
select @LNeed=@Log_MB/8000
IF (@Log_MB%8000)>=0 
SELECT @LNeed=@LNeed+1 
WHILE (@LCnt<=@LNeed) BEGIN
SET @Log_curr=CASE WHEN @LCnt=1 and @Log_MB<=8000 THEN @Log_MB
WHEN @Log_MB-(8000*@LCnt)>0 THEN 8000 
WHEN @Log_MB-(8000*@LCnt)<0 THEN @Log_MB-(8000*(@LCnt-1))
END				
select @Ac=@Ac+@Log_curr
if @Log_curr>0
select top 1 @T=@T+''ALTER DATABASE [''+d.name+''] MODIFY FILE ( NAME=N''''''+mf.name+'''''', SIZE =''+convert(varchar(1000), @Ac)+'' MB );
GO
''
FROM sys.databases d join sys.master_files mf on d.database_id=mf.database_id where type_desc=''log'' and db_name()=d.name
SELECT @LCnt=@LCnt+1 
END 
END
IF @T IS NOT NULL BEGIN
set @T=@T+''
''
IF @Co>(@Log_MB/100) and @Co>50 SELECT DB_NAME()+'' excessive VLF count.'';
IF @Avg_MB<64 SELECT DB_NAME()+'' VLFs too small'';
IF @Max_MB>=4096 SELECT DB_NAME()+'' single VLF >4GB'';
print @T;
END
Truncate Table #Log;'

Drop Table #Log;


/*
More reference
A VLF is comprised of 1 or more log blocks each of which is an integer multiple of 512 bytes, but no more than 60KB total in size.
----http://www.sqlskills.com/blogs/kimberly/transaction-log-vlfs-too-many-or-too-few/
----http://www.sqlskills.com/blogs/kimberly/8-steps-to-better-transaction-log-throughput/
----https://blogs.msdn.microsoft.com/alwaysonpro/2013/09/27/performing-transaction-log-backups-using-alwayson-availability-group-read-only-secondary-replicas-part-1/
----https://www.red-gate.com/simple-talk/sql/database-administration/sql-server-transaction-log-fragmentation-a-primer/
---"If you need a 2GB log then just create that as one step. 
---If you need a 20GB log, create that as 8GB, then extend it to 16GB and then to 20GB"
--Optimal size for Avg_MB for VLF's is 500MB.

--displays each transaction log size and space used. 
--Dbcc sqlperf (logspace)  --replaced, look for "space in log files.sql"


*/

/*

--Script to test database, create suboptimal VLF's
USE [w]
GO
DBCC SHRINKFILE (N'w2016_log' , 0, TRUNCATEONLY)
GO

USE [master]
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1001MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1002MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1003MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1004MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1005MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1006MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1007MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1008MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1009MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1010MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1011MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1012MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1013MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1014MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1015MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1016MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1017MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1018MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1019MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1020MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1021MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1022MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1023MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1024MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1025MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1026MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1027MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1028MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1029MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1030MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1031MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1032MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1033MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1034MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1035MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1036MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1037MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1038MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1039MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1040MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1041MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1042MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1043MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1044MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1045MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1046MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1047MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1048MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1049MB );
GO
--Won't show up in query above until here
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=1050MB );
GO
--further testing the 8 GB growth pattern
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=8000MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=8001MB );
GO
ALTER DATABASE [w] MODIFY FILE ( NAME=N'w2016_log', SIZE=16001MB );
GO
*/

/*

--Sample usage

USE [Tfs_Warehouse]
DBCC SHRINKFILE (N'Tfs_Warehouse_log' , 0, TRUNCATEONLY);
GO
USE [master]
--Original Size 1024.00 MB
ALTER DATABASE [Tfs_Warehouse] MODIFY FILE ( NAME=N'Tfs_Warehouse_log', SIZE=1024MB );
GO



USE [Tfs_SparkyBox]
DBCC SHRINKFILE (N'Tfs_SparkyBox_log' , 0, TRUNCATEONLY);
GO
USE [master]
--Original Size 356.00 MB
ALTER DATABASE [Tfs_SparkyBox] MODIFY FILE ( NAME=N'Tfs_SparkyBox_log', SIZE=356MB );
GO


USE [Tfs_Configuration]
DBCC SHRINKFILE (N'Tfs_Configuration_log' , 0, TRUNCATEONLY);
GO
USE [master]
--Original Size 1024.00 MB
ALTER DATABASE [Tfs_Configuration] MODIFY FILE ( NAME=N'Tfs_Configuration_log', SIZE=1024MB );
GO



*/
	


In [ ]:
--Database Ownership.sql
--Find database owners that are not desired
PRINT('Database Ownership Output Window')

declare @Desired_DB_owner varchar(255) = 'sa' --'sa' is just an example, change to desired service account, example: domain\accountname

select 
	database_name = d.name
,	principal_name = SUSER_SNAME (d.owner_sid)
,	set_to_desired = 'alter authorization on database::[' + d.name + '] to [' + @Desired_DB_owner + ']' 
,	set_to_current =  case when SUSER_SNAME (d.owner_sid) <> @Desired_DB_owner THEN 'alter authorization on database::[' + d.name + '] to [' + SUSER_SNAME (d.owner_sid) + ']' ELSE NULL END
,	* 
from sys.databases d
where SUSER_SNAME (d.owner_sid) <> @Desired_DB_owner

In [ ]:
--Job Owners.sql
PRINT('Job Owner Output Window')
use msdb
go
--TODO Change @owner_login_name to desired SQL agent service account to own the job

declare @Desired_job_owner varchar(255) = 'SPARKHOUND\svcaccount' --'sa' is just an example, change to desired service account, example: domain\accountname

--sql 2005 and above
select owner = SUSER_SNAME (j.owner_sid), jobname = j.name, j.job_id
,	change_tsql = N'EXEC msdb.dbo.sp_update_job @job_id=N'''+convert(nvarchar(100), j.job_id)+N''', @owner_login_name=N'''+@Desired_job_owner+''''
,	revert_tsql = N'EXEC msdb.dbo.sp_update_job @job_id=N'''+convert(nvarchar(100), j.job_id)+N''', @owner_login_name=N'''+SUSER_SNAME (j.owner_sid)+''''
from sysjobs j
left outer join sys.server_principals  sp on j.owner_sid = sp.sid
where 
	(sp.name not in ('sa','distributor_admin','NT SERVICE\ReportServer') 
	 and sp.name <> @Desired_job_owner
	 and sp.name not like '##%')
	or sp.name is null 

/*
--sql 2000
select sp.name, j.name, j.job_id from msdb.dbo.sysjobs j
left outer join master.dbo.syslogins sp on j.owner_sid = sp.sid
where sp.name not in ('sa','distributor_admin') or sp.name is null
--EXEC msdb.dbo.sp_update_job @job_id=N'8eab379e-958e-4576-92ae-b5999aeec01c', @owner_login_name=N'distributor_admin'
*/

/*
--Sample usage

EXEC msdb.dbo.sp_update_job @job_id=N'BDAFAC9B-1705-4E47-9C26-6C4B813CB165', @owner_login_name=N'sa'
EXEC msdb.dbo.sp_update_job @job_id=N'987AF666-A516-4847-8BA3-73DE337CFF94', @owner_login_name=N'sa'
EXEC msdb.dbo.sp_update_job @job_id=N'AEBF4C5C-EC6D-4635-96B6-797BF4AEEC62', @owner_login_name=N'sa'

*/

In [ ]:
--Tempdb Data Files.sql
--TempDB data files should all be the same size, same autogrowth settings
--Only displays TempDB data files.
PRINT('TempDB Data Files Output Window')
USE tempdb
GO
DECLARE @cpu_count int;
SELECT @cpu_count = cpu_count from sys.dm_os_sys_info;

--data files
select mf.name
, CurrentSize_MB = (d.size*8.)/1024. --actual current file size
, InitialSize_MB = (mf.size*8.)/1024. --initial file size
, GrowthMb = (mf.growth*8.)/1024.
, mf.is_percent_growth
, MaxFileSizeMB = CASE WHEN mf.max_size > -1 THEN cast((mf.max_size*8.)/1024. as varchar(100)) ELSE 'Unlimited' END -- "-1" is unlimited
, Recommendation = CASE WHEN d.size > mf.size THEN 'Increase TempDB Data files size to match or exceed current size.' + CHAR(10) ELSE '' END +
CASE WHEN mf.size <> AVG(mf.size) OVER (PARTITION BY mf.database_id) THEN 'Set all TempDB Data files to the same initial size.' + CHAR(10) ELSE '' END + 
CASE WHEN d.size <> AVG(d.size) OVER (PARTITION BY mf.database_id) THEN 'Grow TempDB Data files to the same current size.' + CHAR(10) ELSE '' END +
CASE WHEN mf.growth <> AVG(mf.growth) OVER (PARTITION BY mf.database_id) THEN 'Match all TempDB Data files autogrowth rates.' + CHAR(10) ELSE '' END +
CASE WHEN mf.max_size <> AVG(mf.max_size) OVER (PARTITION BY mf.database_id) THEN 'Match all TempDB Data files max file size.' + CHAR(10) ELSE '' END +
CASE WHEN count(mf.file_id) OVER (PARTITION BY mf.database_id) > @cpu_count THEN 'Too many TempDB Data files, reduce to '+cast(@cpu_count as varchar(3)) + ' or lower.' + CHAR(10) ELSE '' END
, mf.physical_name
, volume_letter = UPPER(vs.volume_mount_point)
, file_system_type
, drive_size_GB = (CONVERT(decimal(19,2), vs.total_bytes/1024./1024./1024. ))
, drive_free_space_GB = (CONVERT(decimal(19,2), vs.available_bytes/1024./1024./1024. ))
, drive_pct_free = (CONVERT(DECIMAL(5,2), vs.available_bytes * 100.0 / vs.total_bytes))
from sys.master_files mf
inner join tempdb.sys.database_files d
on mf.file_id = d.file_id
and mf.database_id = db_id()
cross apply sys.dm_os_volume_stats(mf.database_id, mf.file_id) vs --only return volumes where there is database file (data or log)
where d.type_desc = 'rows'
order by mf.file_id asc

--log file
select mf.name
, CurrentSize_MB = (d.size*8.)/1024. --actual current file size
, InitialSize_MB = (mf.size*8.)/1024. --initial file size
, GrowthMb = (mf.growth*8.)/1024.
, mf.is_percent_growth
, MaxFileSizeMB = CASE WHEN mf.max_size > -1 THEN cast((mf.max_size*8.)/1024. as varchar(100)) ELSE 'Unlimited' END -- "-1" is unlimited
, Recommendation = CASE WHEN d.size > mf.size THEN 'Increase TempDB Log file size to match or exceed current size.' + CHAR(10) ELSE '' END
, mf.physical_name
, volume_letter = UPPER(vs.volume_mount_point)
, file_system_type
, drive_size_GB = (CONVERT(decimal(19,2), vs.total_bytes/1024./1024./1024. ))
, drive_free_space_GB = (CONVERT(decimal(19,2), vs.available_bytes/1024./1024./1024. ))
, drive_pct_free = (CONVERT(DECIMAL(5,2), vs.available_bytes * 100.0 / vs.total_bytes))
from sys.master_files mf
inner join tempdb.sys.database_files d
on mf.file_id = d.file_id
and mf.database_id = db_id() 
cross apply sys.dm_os_volume_stats(mf.database_id, mf.file_id) vs --only return volumes where there is database file (data or log)
where d.type_desc = 'log'
order by mf.file_id asc;

SELECT servicename, status_desc, last_startup_time FROM sys.dm_server_services;
GO
--To resize tempdb: 

/*
USE [master]
GO
ALTER DATABASE [tempdb]   MODIFY FILE ( NAME = N'<tempdfilename>'  , SIZE = <Desired File Size>);
GO

USE [tempdb]
GO
CHECKPOINT
GO
DBCC SHRINKFILE (N'<tempdfilename>' , 0, TRUNCATEONLY);
GO
*/


In [ ]:
--Stats Out of Date whileloop.sq
--This script only works in SQL2008R2SP2+ or SQL2012SP1+. 

--This generates an UPDATE STATISTICS script for all databases. Can be used in a maintenance plan (see bottom). Safe to execute. 
--This version does not work in Azure SQL DB. Instead, see toolbox\stats out of date.sql.
--Use toolbox\stats out of date.sql to examine a particular database.

--TODO BEFORE EXECUTING: comment out three lines below in <SQL2014 because incremental stastics not supported.
PRINT('Stats out of Date Output Window')
declare @tsql nvarchar(max) = 
 N'use [?];
    SELECT distinct
--			s.name AS SchemaName 
--          , o.name AS ObjectName 
--          , STA.name AS StatName 
--		  , Object_Type = ISNULL(i.type_desc + '' Index'', ''Statistics Object'')
--          , Stats_Last_Updated = ISNULL(sp.last_updated, o.create_date)
--		  , Rows_Changed = ISNULL(sp.modification_counter,0) --Rows Changed since last update
			--Below block only works in SQL 2014+, comment out this line in prior versions.
			   /*
		  , PartitionNumber = CASE WHEN MAX(p.partition_number) OVER (PARTITION by STA.name, i.name)  > 1 THEN p.partition_number ELSE null END
		  , STA.is_incremental --Only works in SQL 2014+, comment out this line in prior versions.
		  */
--		  , ''use [?]'' AS [?],
		   TSQL = CASE WHEN i.type_desc like ''%columnstore%'' THEN NULL ELSE
		    N''USE [?]; '' + 
			N''UPDATE STATISTICS '' 
               + QUOTENAME(s.name) + N''.'' + QUOTENAME(o.name) + N'' '' 
               + QUOTENAME(STA.name) + N'' '' 
			   + ''WITH RESAMPLE''
			   --Below block only works in SQL 2014+, comment out this line in prior versions.
			   /*
			   + CASE WHEN 
						STA.Is_Incremental = 1 and  --Only works in SQL 2014+, comment out this line in prior versions.
						MAX(p.partition_number) OVER (PARTITION by STA.name, i.name)  > 1 THEN '' ON PARTITIONS ('' + cast(p.partition_number as varchar(5)) + '') '' ELSE '''' END
               */
			END
   FROM sys.objects  o   
		 INNER JOIN sys.stats STA ON STA.object_id = o.object_id  
			CROSS APPLY sys.dm_db_stats_properties (STA.object_id, STA.stats_id) sp -- Only works in SQL2008R2SP2+ or SQL2012SP1+
         INNER JOIN sys.schemas AS s 
             ON o.schema_id = s.schema_id 
		 LEFT OUTER JOIN sys.indexes as i
			on i.index_id = STA.stats_id
			and (i.type_desc not like ''%columnstore%'')
			--Below block only works in SQL 2014+, comment out this line in prior versions.
			 /*
	     LEFT OUTER join sys.dm_db_partition_stats p 
			on (
			STA.Is_Incremental = 1 and  --Only works in SQL 2014+, comment out this line in prior versions. 
			p.object_id = o.object_id  and 
			i.index_id = p.index_id
			)
			*/
         LEFT JOIN 
         (SELECT IUS.object_id 
                ,MIN(ISNULL(IUS.last_user_update, IUS.last_system_update)) AS LastUpdate 
          FROM sys.dm_db_index_usage_stats AS IUS 
          WHERE database_id = DB_ID() 
                AND NOT ISNULL(IUS.last_user_update, IUS.last_system_update) IS NULL 
          GROUP BY IUS.object_id 
         ) AS IUS 
             ON IUS.object_id = STA.object_id 
    WHERE o.type IN (''U'', ''V'')    -- only user tables and views 
          AND DATEDIFF(d, ISNULL(STATS_DATE(STA.object_id, STA.stats_id), N''1900-01-01'')  , IUS.LastUpdate) > 30 --indexes that haven''t been updated in the last month
		  AND sp.modification_counter > 10000
    OPTION (MAXDOP 1);
	print ''[?]'' '

declare @dblist table (id int not null identity(1,1) primary key, dbname sysname not null)
declare @tsqllist table (id int not null identity(1,1) primary key, tsqltext nvarchar(4000) not null) 
declare @x int = 1, @xmax int = null, @dbname sysname, @runtsql nvarchar(max) = null
insert into @dblist 
select name from sys.databases
where state_desc = 'online' and (database_id > 4 or name = 'msdb')
select @xmax = max(id) from @dblist l

while (@x <= @xmax)
BEGIN
	select @dbname = dbname from @dblist l where @x = id

	select @runtsql = replace(@tsql, N'?', @dbname)

	--generates scripts, does not actually perform the UPDATE STATISTICS. See below.
	--Writes all the TSQL into a table variable which is displayed later
	insert into @tsqllist (tsqltext) 
	exec sp_executesql @runtsql --safe, does not actually update stats
	
	set @x = @x + 1
END

--Shows all stats in all databases that need to be updated
select * from @tsqllist

--OPTIONALLY - execute all UPDATE Stats
/*
declare @s int = 1, @scount int = null
select @scount = max(id), @runtsql = null from @tsqllist l
while (@s <= @scount)
BEGIN
	
	--actually executes the scripts.
	select @runtsql = tsqltext from @tsqllist where id = @s
	exec sp_executesql @runtsql
	
	set @s = @s + 1
	
END
*/




/*

USE [WideWorldImporters]; UPDATE STATISTICS [Sales].[InvoiceLines] [_WA_Sys_0000000D_1E6F845E] WITH RESAMPLE
USE [WideWorldImporters]; UPDATE STATISTICS [Sales].[Invoices] [_WA_Sys_0000000A_7849DB76] WITH RESAMPLE

*/


In [ ]:
--last known DBCC CHECKDB.sql
PRINT('Last Known Good DBCC CheckDB')
EXEC sp_MSforeachdb '
--Table variable to capture the DBCC DBINFO output, look for the field we want in each database output
DECLARE @DBCC_DBINFO TABLE (ParentObject VARCHAR(255) NOT NULL, [Object] VARCHAR(255)  NOT NULL, [Field] VARCHAR(255) NOT NULL 
INDEX idx_dbinfo_field CLUSTERED --just this line is SQL 2014+ only
, [Value] VARCHAR(255));
INSERT INTO @DBCC_DBINFO EXECUTE ("DBCC DBINFO ([?]) WITH TABLERESULTS");
SELECT DISTINCT ''?'', [Value] FROM @DBCC_DBINFO WHERE Field = ''dbi_dbccLastKnownGood'';';


In [ ]:
--Volume Stats.sql
PRINT('Volume Stats Output Window')
select 
  volume_letter = UPPER(vs.volume_mount_point)
, volume_name = vs.logical_volume_name
, file_system_type
, drive_size_GB = MAX(CONVERT(decimal(19,2), vs.total_bytes/1024./1024./1024. ))
, drive_free_space_GB = MAX(CONVERT(decimal(19,2), vs.available_bytes/1024./1024./1024. ))
, drive_percent_free = MAX(CONVERT(decimal(5,2), vs.available_bytes * 100.0 / vs.total_bytes))
FROM
   sys.master_files AS f CROSS APPLY
   sys.dm_os_volume_stats(f.database_id, f.file_id) vs --only return volumes where there is database file (data or log)
 GROUP BY vs.volume_mount_point, vs.file_system_type, vs.logical_volume_name
 ORDER BY volume_letter 

Disaster Recovery
1.	Review current backup status, look for missing FULL or missing TLOG backups for DB’s not in SIMPLE.
a.	Backup history.sql
2.	Retention settings
a.	If local backups are being taken, view the backup folder and verify that retention policy is deleting old backups.
b.	Verify that backup chains are intact - .bak’s should not be deleted before dependent .dif and .trn files.


In [ ]:
--Backup hisotry.sql
--Looks for a complete backup history, displaying the latest backup of each type.

--See also toolbox/multiserver backup history.sql for health checks.
PRINT('Backup History Output Window')
use master
go
--sql2012 and above
select 
	database_Name
	, backuptype 
	, d.recovery_model_desc
	, BackupDate = MAX(BackupDate)
	, d.state_desc
	, d.is_read_only
	, dm.Replica_Role		--SQL 2012+
 from sys.databases d
 inner join 
 (
select distinct 
	database_name
	, database_id = db_id(database_name)
	, backuptype = case type	WHEN 'D' then 'Database'
							WHEN 'I' then 'Differential database'
							WHEN 'L' then 'Transaction Log'
							WHEN 'F' then 'File or filegroup'
							WHEN 'G' then 'Differential file'
							WHEN 'P' then 'Partial'
							WHEN 'Q' then 'Differential partial' END
	, BackupDate	=	MAX(backup_finish_date)  	
	from msdb.dbo.backupset bs							
 group by Database_name, type
 UNION 
 select distinct
	db_name(d.database_id)
	, d.database_id
	, backuptype = 'Database'
	, null
	FROM master.sys.databases d
 UNION
 select distinct
	db_name(d.database_id)
	, d.database_id
	, backuptype = 'Transaction Log'
	, null
  FROM master.sys.databases d
  where d.recovery_model_desc in ('FULL', 'BULK_LOGGED')
 ) a
 on d.database_id = a.database_id
 
 --SQL 2012+
 LEFT OUTER JOIN ( SELECT  database_id 
						 ,	Replica_Role		= CASE WHEN database_state_desc IS NOT NULL and last_received_time is null THEN 'PRIMARY '
															WHEN database_state_desc IS NOT NULL and last_received_time is not null THEN 'SECONDARY' 
															ELSE null END
															from sys.dm_hadr_database_replica_states) dm
						 on dm.database_id = a.database_id
WHERE database_name not in ('model','tempdb')
and not (backuptype = 'transaction log' and recovery_model_desc = 'SIMPLE')
group by database_name, backuptype, d.recovery_model_desc, d.state_desc, d.is_read_only, dm.replica_role
order by backuptype, recovery_model_desc, database_name asc
go

 /*
--for SQL 2000 and above
select distinct 
	  database_name	= d.name 
	, a.backuptype	
	, RecoveryModel	=	databasepropertyex(d.name, 'Recovery')  
	, BackupDate	=	Max(a.backup_finish_date)  
	from master.dbo.sysdatabases d
	left outer join 
	(		select distinct 
			database_name
			, backuptype = case type	WHEN 'D' then 'Database'
									WHEN 'I' then 'Differential database backup'
									WHEN 'L' then 'Transaction Log'
									WHEN 'F' then 'File or filegroup'
									WHEN 'G' then 'Differential file'
									WHEN 'P' then 'Partial'
									WHEN 'Q' then 'Differential partial' END
			, backup_finish_date	=	MAX(backup_finish_date)  	
			from msdb.dbo.backupset bs							
		 group by Database_name, type
		 UNION 
		 select distinct
			  d.name
			, backuptype = 'Database'
			, null
			FROM master.dbo.sysdatabases d
		 UNION
		 select distinct
			  d.name
			, backuptype = 'Transaction Log'
			, null
		  FROM master.dbo.sysdatabases d
		  where databasepropertyex(d.name, 'Recovery') in ('FULL', 'BULK_LOGGED')
  ) a
	on d.name = a.database_name
 group by d.name , backuptype ,	databasepropertyex(d.name, 'Recovery')
order by backuptype, RecoveryModel, BackupDate asc
 */
 
--granular backup history
SELECT 
		bs.database_name
	, backuptype = CASE 
							WHEN bs.type = 'D' and bs.is_copy_only = 0 then 'Full Database'
							WHEN bs.type = 'D' and bs.is_copy_only = 1 then 'Full Copy-Only Database'
							WHEN bs.type = 'I' then 'Differential database backup'
							WHEN bs.type = 'L' then 'Transaction Log'
							WHEN bs.type = 'F' then 'File or filegroup'
							WHEN bs.type = 'G' then 'Differential file'
							WHEN bs.type = 'P' then 'Partial'
							WHEN bs.type = 'Q' then 'Differential partial' END + ' Backup'
	, bs.recovery_model
	, BackupStartDate = bs.Backup_Start_Date
	, BackupFinishDate = bs.Backup_Finish_Date
	, LatestBackupLocation = bf.physical_device_name
	, backup_size_mb			=	bs.backup_size / 1024./1024.
	, compressed_backup_size_mb =	bs.compressed_backup_size /1024./1024.
	, database_backup_lsn -- For tlog and differential backups, this is the checkpoint_lsn of the FULL backup it is based on. 
	, checkpoint_lsn
	, begins_log_chain
	FROM msdb.dbo.backupset bs	
	LEFT OUTER JOIN msdb.dbo.[backupmediafamily] bf
	on bs.[media_set_id] = bf.[media_set_id]
	--where database_name = 'w'
	ORDER BY  bs.database_name asc, bs.Backup_Start_Date desc;
 

 
 /*
  --Latest Restore
 select d.name, Latest_Restore = max(restore_date)
	from sys.databases d
	LEFT OUTER JOIN msdb.dbo.restorehistory rh on d.name = rh.destination_database_name
	group by d.name
	order by Latest_Restore desc

*/


--Look for backups to NUL, a sign that someone doesn't know what they're doing to the tlog. (Probably VEEAM. Bad VEEAM.)
--This is bad. Backup to NUL is just truncating the log without backing up the log, breaking the tlog chain. Any subsequent tlog backups are broken until a FULL backup restarts a valid chain.
--Do not allow VEEAM or other VSS-based backup solutions to do backups to NUL. 
--In VEEAM, this is somewhere near the "application aware backups" or similar settings menu in various settings. Disable this. 
SELECT 
	  bs.database_name
	, backuptype = CASE 
							WHEN bs.type = 'D' and bs.is_copy_only = 0 then 'Full Database'
							WHEN bs.type = 'D' and bs.is_copy_only = 1 then 'Full Copy-Only Database'
							WHEN bs.type = 'I' then 'Differential database backup'
							WHEN bs.type = 'L' then 'Transaction Log'
							WHEN bs.type = 'F' then 'File or filegroup'
							WHEN bs.type = 'G' then 'Differential file'
							WHEN bs.type = 'P' then 'Partial'
							WHEN bs.type = 'Q' then 'Differential partial' END + ' Backup'
	, bs.recovery_model
	, BackupStartDate = bs.Backup_Start_Date
	, BackupFinishDate = bs.Backup_Finish_Date
	, LatestBackupLocation = bf.physical_device_name
	, backup_size_mb			=	bs.backup_size / 1024./1024.
	, compressed_backup_size_mb =	bs.compressed_backup_size /1024./1024.
	, database_backup_lsn -- For tlog and differential backups, this is the checkpoint_lsn of the FULL backup it is based on. 
	, checkpoint_lsn
	, begins_log_chain
	FROM msdb.dbo.backupset bs	
	LEFT OUTER JOIN msdb.dbo.[backupmediafamily] bf
	on bs.[media_set_id] = bf.[media_set_id]
	where bf.physical_device_name = 'NUL'
	ORDER BY  bs.database_name asc, bs.Backup_Start_Date desc;


Database Mail
1.	View Database Mail log for failures
a.	Look for unsent mails, unstarted broker, stopped process in Database Mail Diag.sql
2.	Verify functionality by sending a test email to managed.sql@sparkhound.com or appropriate distribution group. 


In [ ]:
--Database Mail Diag.sql
PRINT('Databse Mail Diagnostic Output Window')
SELECT is_broker_enabled FROM sys.databases WHERE name = 'msdb' ; -- should be 1
EXECUTE msdb.dbo.sysmail_help_status_sp ; --should say STARTED
--EXECUTE msdb.dbo.sysmail_start_sp --start the database mail queues;
GO

--Find recent unsent emails, hopefully there are none
SELECT m.send_request_date, m.recipients, m.copy_recipients, m.blind_copy_recipients
, m.[subject], sent_account = a.name, m.send_request_user, m.sent_status
, Error_Description = l.description 
FROM msdb.dbo.sysmail_allitems m
LEFT OUTER JOIN msdb.dbo.sysmail_account a
	ON m.sent_account_id = a.account_id
LEFT OUTER JOIN msdb.dbo.sysmail_event_log AS l  
    ON m.mailitem_id = l.mailitem_id  
WHERE	1=1
AND     m.send_request_date > dateadd(day, -45, sysdatetime()) -- Only show recent day(s)
AND		m.sent_status <> 'sent' -- Possible values are sent (successful), unsent (in process), retrying (failed but retrying), failed (no longer retrying)
ORDER BY m.send_request_date DESC;
GO

--Send mail test
--exec msdb.dbo.sp_send_dbmail @profile_name ='hotmail', @recipients ='williamdassaf@hotmail.com', @subject ='test', @body = 'test'

--ALTER DATABASE msdb SET ENABLE_BROKER;

Security
1.	Run Public Permissions.sql to identify any SELECT and/or EXECUTE permissions granted to the public server role.
a.	Do not recommend granting ALTER, CONTROL, REFERENCES, TAKE OWNERSHIP, VIEW DEFINITION, to anyone who is not already an admin. We should be able to revoke this permission from public with no replacement unless the application is doing something with dynamically creating/altering objects. These permissions shouldn’t be needed by anyone not doing deployments of code changes and shouldn’t be necessary for day-to-day operation unless the application is doing something with dynamically creating/altering objects.
b.	Need developer opinions on how DELETEs, INSERTs, UPDATEs are actually made. Then, grant these DELETE, INSERT or UPDATE permissions to something that is clearly not a read-only user, such as an application service user.
c.	It is not uncommon to grant EXECUTE for a whole database to an application or app service user, but it should not be granted to a read-only user. Perhaps there is a subset of sprocs that are for reporting that a read-only user should have access to EXECUTE, would need developer insight on that.
d.	SELECT is the easiest topic. 
i.	There is a SELECT permission granted to public on the entire [dbo] schema. All the individual GRANT SELECT on dbo objects can be revoked and replaced by a single GRANT SELECT ON SCHEMA::[dbo] to one or more application service users including read-only users.
ii.	For GRANT SELECT on objects on other non-dbo schemas, these could be used by read-write application service users and read-only service users. We should replace the public permission with GRANT SELECT on these objects to any user that could access these tables.


In [ ]:
--Public Permissions.sql
PRINT('Public Permissions Output Window')
SELECT @@SERVERNAME


--Server Level Security
SELECT rm.state_desc, rm.permission_name, principal_name = QUOTENAME(u.name),  u.type_desc
,  TSQL = rm.state_desc + N' ' + rm.permission_name + 
	CASE WHEN e.name is not null THEN ' ON ENDPOINT::[' + e.name + '] ' ELSE '' END +
	N' TO ' + cast(QUOTENAME(u.name COLLATE DATABASE_DEFAULT) as nvarchar(256)) + ';'
,  TSQL = N'REVOKE ' + rm.permission_name +
	CASE WHEN e.name is not null THEN ' ON ENDPOINT::[' + e.name + '] ' ELSE '' END +
	 N' TO ' + cast(QUOTENAME(u.name COLLATE DATABASE_DEFAULT) as nvarchar(256)) + ';', *
FROM sys.server_permissions rm
inner join sys.server_principals u 
on rm.grantee_principal_id = u.principal_id
left outer join sys.endpoints e
on e.endpoint_id = major_id and class_desc = 'ENDPOINT'
where u.name not like '##%' 
and u.name = 'public'
order by rm.permission_name, u.name


--Database role membership
--Multi Database
declare @TSQL nvarchar(4000) = 'use [?]; 
SELECT DB_NAME();
SELECT DISTINCT	QUOTENAME(r.name) as database_role_name, r.type_desc, QUOTENAME(d.name) as principal_name, d.type_desc
,	Add_TSQL = ''EXEC sp_addrolemember @membername = N'''''' + d.name COLLATE DATABASE_DEFAULT + '''''', @rolename = N'''''' + r.name + ''''''''
,	Drop_TSQL = ''EXEC sp_droprolemember @membername = N'''''' + d.name COLLATE DATABASE_DEFAULT + '''''', @rolename = N'''''' + r.name + ''''''''
FROM	sys.database_role_members rm
inner join sys.database_principals r on rm.role_principal_id = r.principal_id
inner join sys.database_principals d on rm.member_principal_id = d.principal_id
where d.name = ''public''
';
EXEC sp_MSforeachdb @TSQL
GO

--Multi-database database permissions
--script is too long for sp_msforeachdb, had to roll our own.

declare @TSQL varchar(8000) = null, @dbcount int = 0, @x int = 0, @dbname varchar(256) = null
declare @dblist table (id int not null identity(1,1) primary key, dbname varchar(256)  not null )
insert into @dblist (dbname)
select name from sys.databases where name <> 'tempdb' and state_desc = 'ONLINE' 
order by database_id
select @dbcount = count(dbname) from @dblist

while (@x <= @dbcount)
BEGIN
	select @dbname = dbname from @dblist d where @x = d.id;

	select @TSQL = 	'USE [' + @dbname  + '];
	SELECT DB_NAME();
	SELECT	Permission_State_Desc	=	perm.state_desc
		,	Permission_Name			=	perm.permission_name 
		,	Permission_Object_Name	= ISNULL(QUOTENAME(s.name ) + ''.'','''') + QUOTENAME(obj.name COLLATE database_default) + CASE WHEN cl.name COLLATE database_default is null THEN '''' ELSE ''.'' + QUOTENAME(cl.name COLLATE database_default) END 			
		,	Object_Type_Desc		=	obj.type_desc   
		,	Principal_Name			=	QUOTENAME(u.name COLLATE database_default) 
		,	User_Type				=	u.type_desc 
		,	Create_TSQL = perm.state_desc + N'' '' + perm.permission_name 
			+ case when obj.name COLLATE database_default is not null THEN + N'' ON '' + sc.class_desc + ''::'' + ISNULL(QUOTENAME(s.name COLLATE database_default) + ''.'','''') + QUOTENAME(obj.name COLLATE database_default) ELSE '''' END 
			+ CASE WHEN cl.column_id IS NULL THEN '' '' ELSE ''('' + QUOTENAME(cl.name COLLATE database_default) + '')'' END 
			+ N'' TO '' + QUOTENAME(u.name COLLATE database_default)
		,	Revoke_TSQL = N''REVOKE '' + perm.permission_name 
			+ case when obj.name COLLATE database_default is not null THEN + N'' ON '' + sc.class_desc + ''::'' + ISNULL(QUOTENAME(s.name COLLATE database_default) + ''.'','''') + QUOTENAME(obj.name COLLATE database_default) ELSE '''' END 
			+ CASE WHEN cl.column_id IS NULL THEN '' '' ELSE ''('' + QUOTENAME(cl.name COLLATE database_default) + '')'' END 
			+ N'' TO '' + QUOTENAME(u.name COLLATE database_default) 
			, *
	FROM sys.database_permissions AS perm 
	INNER JOIN sys.database_principals AS u	ON perm.grantee_principal_id = u.principal_id
	LEFT OUTER JOIN (--https://msdn.microsoft.com/en-us/library/ms188367.aspx			
						select name, object_id, schema_id, is_ms_shipped, class_desc=''OBJECT'', type_desc from sys.objects 
						union all
						select name, 0, null, null, ''DATABASE'', ''DATABASE''  from sys.databases 	
						union all
						select  name, schema_id, null, null, ''SCHEMA'', ''SCHEMA'' from sys.schemas
						union all
						select name, principal_id, null, null,  ''USER'', type_desc from sys.database_principals where type_desc in (''WINDOWS_USER'',''SQL_USER'',''ASYMMETRIC_KEY_MAPPED_USER'',''CERTIFICATE_MAPPED_USER'', ''WINDOWS_GROUP'',''EXTERNAL_GROUPS'')
						union all
						select name, principal_id, null, null,  ''USER'', type_desc from sys.database_principals where type_desc in (''WINDOWS_USER'',''SQL_USER'',''ASYMMETRIC_KEY_MAPPED_USER'',''CERTIFICATE_MAPPED_USER'', ''WINDOWS_GROUP'',''EXTERNAL_GROUPS'')
						union all
						select name, principal_id, null, null, ''APPLICATION ROLE'', type_desc from sys.database_principals where type_desc in (''APPLICATION_ROLE'')
						union all
						select name, principal_id, null, null, ''ROLE'', type_desc from sys.database_principals where type_desc in (''DATABASE_ROLE'')
						union all
						select name, assembly_id, null, null, ''ASSEMBLY'', ''ASSEMBLY'' from sys.assemblies 
						union all
						select name, user_type_id, null, null, ''TYPE'', ''USER TYPE'' from sys.types 
						union all
						select name, xml_collection_id, null, null, ''XML SCHEMA COLLECTION'', ''XML SCHEMA COLLECTION'' from sys.xml_schema_collections
						union all
						select name COLLATE database_default, message_type_id, null, null, ''MESSAGE TYPE'', ''MESSAGE TYPE'' from sys.service_message_types
						union all
						select name COLLATE database_default, service_contract_id, null, null, ''CONTRACT'', ''CONTRACT'' from sys.service_contracts
						union all
						select name COLLATE database_default, service_id, null, null, ''SERVICE'', ''SERVICE'' from sys.services
						union all
						select name COLLATE database_default, remote_service_binding_id, null, null, ''REMOTE SERVICE BINDING'', ''REMOTE SERVICE BINDING'' from sys.remote_service_bindings
						union all
						select name COLLATE database_default, route_id, null, null, ''ROUTE'', ''ROUTE''  from sys.routes
						union all
						select name COLLATE database_default, fulltext_catalog_id, null, null, ''FULLTEXT CATALOG'', ''FULLTEXT CATALOG''  from sys.fulltext_catalogs
						union all
						select name, symmetric_key_id, null, null, ''SYMMETRIC KEY'', ''SYMMETRIC KEY''  from sys.symmetric_keys
						union all
						select name, certificate_id, null, null, ''CERTIFICATE'', ''CERTIFICATE'' from sys.certificates
						union all
						select name, asymmetric_key_id, null, null, ''ASYMMETRIC KEY'', ''ASYMMETRIC KEY'' from sys.asymmetric_keys
				) obj
	ON perm.major_id = obj.[object_id] 
	INNER JOIN sys.securable_classes sc on sc.class = perm.class 
	and sc.class_desc = obj.class_desc
	LEFT OUTER JOIN sys.schemas s ON s.schema_id = obj.schema_id
	LEFT OUTER JOIN sys.columns cl ON cl.column_id = perm.minor_id AND cl.[object_id] = perm.major_id
	where 1=1
	and u.name = ''public''
	and perm.major_id > 0
	--Ignore internal principals
	and u.name COLLATE database_default not in (''dbo'',''guest'',''INFORMATION_SCHEMA'',''sys'',''MS_DataCollectorInternalUser'',''PolicyAdministratorRole'',''ServerGroupReaderRole''
	,''ServerGroupAdministratorRole'',''TargetServersRole'',''SQLAgentUserRole'',''UtilityCMRReader'',''SQLAgentOperatorRole'',''dc_operator'',''dc_proxy'',''dc_admin'',''db_ssisadmin'',''db_ssisltduser'',''db_ssisoperator''
	,''UtilityIMRWriter'',''UtilityIMRReader'',''RSExecRole'',''DatabaseMailUserRole'')
	--Ignore ## principals
	and u.name COLLATE database_default not like ''##%##''
	--Ignore built-in svc accounts (not recommended anyway!)
	and u.name COLLATE database_default not like ''NT SERVICE%''
	--Ignore MS shipped internal objects 
	and (obj.is_ms_shipped = 0 or obj.is_ms_shipped is null) 
	--Ignore system sprocs (be aware of your naming conventions!)
	--and (obj.name not like ''sp_%'' or obj.name is null)
	--Ignore SSMS Diagramming Objects
	AND (ISNULL(QUOTENAME(s.name ) + ''.'','''') + QUOTENAME(obj.name COLLATE database_default) + CASE WHEN cl.name COLLATE database_default is null THEN '''' ELSE ''.'' + QUOTENAME(cl.name COLLATE database_default) END)
	NOT IN (''[dbo].[fn_diagramobjects]'',
			''[dbo].[sp_helpdiagrams]'',
			''[dbo].[sp_helpdiagramdefinition]'',
			''[dbo].[sp_creatediagram]'',
			''[dbo].[sp_renamediagram]'',
			''[dbo].[sp_alterdiagram]'',
			''[dbo].[sp_dropdiagram]'')
	--Ignore Database Tuning Advisor Objects
	AND (ISNULL(QUOTENAME(s.name ) + ''.'','''') + QUOTENAME(obj.name COLLATE database_default) + CASE WHEN cl.name COLLATE database_default is null THEN '''' ELSE ''.'' + QUOTENAME(cl.name COLLATE database_default) END)
	NOT IN (''[dbo].[dt_generateansiname]'',
			''[dbo].[dt_adduserobject]'',
			''[dbo].[dtproperties]'',
			''[dbo].[dt_setpropertybyid]'',
			''[dbo].[dt_getobjwithprop]'',
			''[dbo].[dt_getpropertiesbyid]'',
			''[dbo].[dt_setpropertybyid_u]'',
			''[dbo].[dt_getobjwithprop_u]'',
			''[dbo].[dt_getpropertiesbyid_u]'',
			''[dbo].[dt_dropuserobjectbyid]'',
			''[dbo].[dt_droppropertiesbyid]'',
			''[dbo].[dt_verstamp006]'',
			''[dbo].[dt_verstamp007]'',
			''[dbo].[dt_getpropertiesbyid_vcs]'',
			''[dbo].[dt_displayoaerror]'',
			''[dbo].[dt_adduserobject_vcs]'',
			''[dbo].[dt_addtosourcecontrol]'',
			''[dbo].[dt_checkinobject]'',
			''[dbo].[dt_checkoutobject]'',
			''[dbo].[dt_isundersourcecontrol]'',
			''[dbo].[dt_removefromsourcecontrol]'',
			''[dbo].[dt_validateloginparams]'',
			''[dbo].[dt_vcsenabled]'',
			''[dbo].[dt_whocheckedout]'',
			''[dbo].[dt_getpropertiesbyid_vcs_u]'',
			''[dbo].[dt_displayoaerror_u]'',
			''[dbo].[dt_addtosourcecontrol_u]'',
			''[dbo].[dt_checkinobject_u]'',
			''[dbo].[dt_checkoutobject_u]'',
			''[dbo].[dt_isundersourcecontrol_u]'',
			''[dbo].[dt_validateloginparams_u]'',
			''[dbo].[dt_whocheckedout_u]'')
	order by Object_Type_Desc, Principal_Name';


	exec (@TSQL);
	select @x = @x + 1;
END



High Availability
1.	If in a Failover Cluster or Availability Group, check the Failover Cluster Manager application in Windows. Report any recent cluster events.
2.	Cluster Nodes must be identical SQL Server version.
3.	If in an Availaiblity Group, use AG - Monitor.sql. Check the Availability Group node statuses and verify are all online and that all secondaries are SYNCHRONIZED (if synchronous) or SYNCHRONIZING (if asynchronous.) 
4.	After re-creating an availability group, the owner will be the login that performed the setup. Microsoft recommends changing the availability group owner to sa. The Availability Group may fail if the owner sid is removed from SQL Server.
a.	AG – check owner.sql
b.	ALTER AUTHORIZATION ON AVAILABILITY GROUP::AGname to [sa] ;
5.	Trend and Baseline for the following counters
a.	Perfmon: AG – Perfmon Counters.sql 
6.	For multisubnet AG’s, verify via nslookup that the Listener is broadcasting an IP in each subnet.
a.	Example: nslookup listenername
b.	You should see two IP’s in two subnets returned.
7.	All logins used to access SQL Server must be on all the replicas. 
8.	Compare select * from sys.sql_logins on all AG replicas to confirm that login names and sid's match. The sid's will be the same on each replica, because they are replicated with each database. If they do not match, sql logins may not be mapped to database users on secondary replicas and connectivity will be denied upon failover. Would need to recreate the logins with the correct SID.


In [ ]:
--AG - Monitor.sql
--On a secondary replica, this view returns a row for every secondary database on the server instance. 
--On the primary replica, this view returns a row for each primary database and an additional row for the corresponding secondary database.

--Updated WDA 20180209


--Monitor Availability Group performance
--On a secondary replica, this view returns a row for every secondary database on the server instance. 
--On the primary replica, this view returns a row for each primary database and an additional row for the corresponding secondary database. Recommended.
PRINT('AG - Monitor Output Window')
IF NOT EXISTS (
SELECT @@SERVERNAME
   FROM sys.dm_hadr_availability_replica_states
   WHERE is_local = 1
   and role_desc = 'PRIMARY'
)
  SELECT 'Recommend: Run This Script on Primary Replica';

DECLARE @BytesFlushed_Start_ms bigint, @BytesFlushed_Start bigint, @BytesFlushed_End_ms bigint, @BytesFlushed_End bigint

DECLARE @TransactionDelay TABLE
(	DB sysname COLLATE SQL_Latin1_General_CP1_CI_AS not null
,	TransactionDelay_Start_ms decimal(19,2) null
,	TransactionDelay_end_ms decimal(19,2) null
,	TransactionDelay_Start decimal(19,2) null
,	TransactionDelay_end decimal(19,2) null
,	MirroredWriteTranspersec_Start_ms decimal(19,2) null
,	MirroredWriteTranspersec_end_ms decimal(19,2) null
,	MirroredWriteTranspersec_Start decimal(19,2) null
,	MirroredWriteTranspersec_end decimal(19,2) null
,	UNIQUE CLUSTERED (DB)
)

INSERT INTO @TransactionDelay (DB, TransactionDelay_Start_ms, TransactionDelay_Start)
select DB = pc.instance_name
,	TransactionDelay_Start_ms = MAX(ms_ticks)
,	TransactionDelay_Start = MAX(convert(decimal(19,2), pc.cntr_value))
from sys.dm_os_sys_info as si
CROSS APPLY sys.dm_os_performance_counters as pc
 where object_name like '%database replica%'
 and counter_name = 'transaction delay' --cumulative transaction delay in ms
 group by pc.instance_name
  
UPDATE t 
SET MirroredWriteTranspersec_Start_ms = t2.MirroredWriteTranspersec_Start_ms
,	MirroredWriteTranspersec_Start = t2.MirroredWriteTranspersec_Start
from @TransactionDelay t
inner join 
(select DB = pc.instance_name
,	MirroredWriteTranspersec_Start_ms = MAX(ms_ticks)
,	MirroredWriteTranspersec_Start = MAX(convert(decimal(19,2), pc.cntr_value))
from sys.dm_os_sys_info as si
CROSS APPLY sys.dm_os_performance_counters as pc
 where object_name like '%database replica%'
 and counter_name = 'mirrored write transactions/sec' --actually a cumulative transactions count, not per sec
 group by pc.instance_name
 ) t2 on t.DB = t2.DB

select @BytesFlushed_Start_ms = MAX(ms_ticks), @BytesFlushed_Start = MAX(cntr_value) --the availability database with the highest Tdata_loss becomes the limiting value for RPO compliance.
from sys.dm_os_sys_info
CROSS APPLY sys.dm_os_performance_counters where counter_name like 'Log Bytes Flushed/sec%'

WAITFOR DELAY '00:00:05' --Adjust sample duration between measurements

UPDATE t 
SET TransactionDelay_end_ms = t2.TransactionDelay_end_ms
,	TransactionDelay_end = t2.TransactionDelay_end
from @TransactionDelay t
inner join 
(select DB = pc.instance_name
,	TransactionDelay_end_ms = MAX(ms_ticks)
,	TransactionDelay_end = MAX(convert(decimal(19,2), pc.cntr_value))
from sys.dm_os_sys_info as si
CROSS APPLY sys.dm_os_performance_counters as pc
 where object_name like '%database replica%'
 and counter_name = 'transaction delay' --cumulative transaction delay in ms
 group by pc.instance_name
 ) t2 on t.DB = t2.DB
 
UPDATE t 
SET MirroredWriteTranspersec_end_ms = t2.MirroredWriteTranspersec_end_ms
,	MirroredWriteTranspersec_end = t2.MirroredWriteTranspersec_end
from @TransactionDelay t
inner join 
(select DB = pc.instance_name
,	MirroredWriteTranspersec_end_ms = MAX(ms_ticks)
,	MirroredWriteTranspersec_end = MAX(convert(decimal(19,2), pc.cntr_value))
from sys.dm_os_sys_info as si
CROSS APPLY sys.dm_os_performance_counters as pc
 where object_name like '%database replica%'
 and counter_name = 'mirrored write transactions/sec'  --actually a cumulative transactions count, not per sec
 group by pc.instance_name
 ) t2 on t.DB = t2.DB

select @BytesFlushed_End_ms =  MAX(ms_ticks), @BytesFlushed_End = MAX(cntr_value) --the availability database with the highest Tdata_loss becomes the limiting value for RPO compliance.
from sys.dm_os_sys_info
CROSS APPLY sys.dm_os_performance_counters where counter_name like 'Log Bytes Flushed/sec%'

declare @LogBytesFlushed decimal(19,2) 
set @LogBytesFlushed = (@BytesFlushed_End - @BytesFlushed_Start) / NULLIF(@BytesFlushed_End_ms - @BytesFlushed_Start_ms,0)

--select * from @TransactionDelay

select 
	AG					= ag.name
,	Instance			= ar.replica_server_name + ' ' + case when is_local = 1 then '(local)' else '' end
,	DB					= db_name(dm.database_id)	
,	Replica_Role		= CASE WHEN last_received_time IS NULL THEN 'PRIMARY (Connections: '+ar.primary_role_allow_connections_desc+')' ELSE 'SECONDARY (Connections: '+ar.secondary_role_allow_connections_desc+')' END
,	Last_received_time
,	Last_commit_time
,	dm.synchronization_state_desc 
,	dm.synchronization_health_desc
,	ar.availability_mode_desc
,	ar.failover_mode_desc
,	Suspended = case is_suspended when 1 then suspend_reason_desc else 'NO' end
,	Redo_queue_size_MB		= convert(decimal(19,2),dm.redo_queue_size/1024.)--KB
,	Redo_rate_MB_per_s		= convert(decimal(19,2),dm.redo_rate/1024.) --KB/s
,	Redo_Time_Left_s_RTO	= convert(decimal(19,2),dm.redo_queue_size*1./NULLIF(dm.redo_rate*1.,0)) --only part of RTO. NULL value on secondary replica indicates no sampled activity.
,	Log_Send_Queue_Size_MB		= convert(decimal(19,2),dm.log_send_queue_size/1024.) 
,	Log_Send_Queue_Bytes_flushed_per_s = NULLIF(@LogBytesFlushed ,0)
,	Log_Send_Queue_Time_Left_s_RPO	= convert(decimal(19,2),dm.log_send_queue_size*1./NULLIF(@LogBytesFlushed ,0)) --Rate. NULL value on secondary replica indicates no sampled activity.
,	Sampled_Transactions_count		= (td.MirroredWriteTranspersec_end - td.MirroredWriteTranspersec_start)  
,	Sampled_Transaction_Delay_ms	= (td.TransactionDelay_end - td.TransactionDelay_start)  
--Transaction Delay numbers will be 0 if there is no synchronous replica for the DB
,	Avg_Sampled_Transaction_Delay_ms_per_s	= convert(decimal(19,2), (td.TransactionDelay_end - td.TransactionDelay_Start) / ((td.TransactionDelay_end_ms - td.TransactionDelay_Start_ms)/1000.))
,	Transactions_per_s	= convert(decimal(19,2), ((td.MirroredWriteTranspersec_end - td.MirroredWriteTranspersec_start) / ((td.MirroredWriteTranspersec_End_ms - td.MirroredWriteTranspersec_Start_ms)/1000.)))
,	dm.secondary_lag_seconds --sql 2016 and above only
,	ar.backup_priority
,	ar.modify_date
,	ar.endpoint_url 
,	ar.read_only_routing_url
,	ar.session_timeout
from sys.dm_hadr_database_replica_states dm
INNER JOIN sys.availability_replicas ar on dm.replica_id = ar.replica_id and dm.group_id = ar.group_id
INNER JOIN sys.availability_groups ag on ag.group_id = dm.group_id
LEFT OUTER JOIN @TransactionDelay td on td.DB = db_name(dm.database_id) --LEFT OUTER in case the sys.dm_os_performance_counters DMV is blank because of a rare issue with SQL startup. Still returns the rest of the data.
ORDER BY
	AG			
,	Instance		
,	DB			
,	Replica_Role

--Current node only, should be run on primary
select 
	wait_type
,	waiting_tasks_count
,	wait_time_ms
,	per_wait_ms = convert(decimal(19,2), (convert(decimal(19,2), wait_time_ms)/ convert(decimal(19,2),waiting_tasks_count) ))
from sys.dm_os_wait_stats where waiting_tasks_count >0 
and wait_type like 'HADR_%_COMMIT'

--SELECT log_reuse_wait_desc FROM sys.databases WHERE name = 'operations'

--Check for suspect pages (hopefully 0 rows returned)
--https://msdn.microsoft.com/en-us/library/ms191301.aspx
SELECT * FROM msdb.dbo.suspect_pages
   WHERE (event_type <= 3);

--Check for autorepair events (hopefully 0 rows returned)
--https://msdn.microsoft.com/en-us/library/bb677167.aspx
select db = db_name(database_id)
,	file_id
,	page_id
,	error_type 
,	page_status
,	modification_time
from sys.dm_hadr_auto_page_repair order by modification_time desc

--Replica status (one row per replica when run on the primary)
select 
	ag.name,
	rcs.replica_server_name,
	rs.last_connect_error_number, rs.last_connect_error_description, rs.last_connect_error_timestamp,
	rs.operational_state_desc,
	rs.recovery_health_desc,
	rs.connected_state_desc,
	rs.role_desc,
	rs.synchronization_health_desc
from sys.dm_hadr_availability_replica_states rs --https://docs.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/sys-dm-hadr-availability-replica-states-transact-sql?view=sql-server-2017
inner join sys.dm_hadr_availability_replica_cluster_states rcs
on rs.replica_id = rcs.replica_id
inner join sys.availability_groups ag 
on ag.group_id = rcs.group_id

--https://msdn.microsoft.com/en-us/library/ff877972(v=sql.110).aspx
--https://msdn.microsoft.com/en-us/library/dn135338(v=sql.110).aspx
--https://blogs.msdn.microsoft.com/psssql/2013/09/23/interpreting-the-counter-values-from-sys-dm_os_performance_counters/
--https://msdn.microsoft.com/en-us/library/ms175048.aspx



In [ ]:
--AG - Check Owner.sql
--should not be owned by named account!
--Run this on EACH replica SQL instance, it may be different.
PRINT('AG- Check Owner Output Window')

SELECT ar.replica_server_name
	,ag.name AS ag_name
	,ar.owner_sid
	,sp.name
FROM sys.availability_replicas ar
LEFT JOIN sys.server_principals sp
	ON sp.sid = ar.owner_sid 
INNER JOIN sys.availability_groups ag
	ON ag.group_id = ar.group_id
WHERE ar.replica_server_name = SERVERPROPERTY('ServerName') ;

/*

ALTER AUTHORIZATION ON AVAILABILITY GROUP::[drAG1] to [sa] ;

*/

In [ ]:
--AG- Perform Counters.sql
PRINT('AG - Performance Counters Output Window')
declare @tempperfmon table (
[object_name]	nchar	(256) null,
counter_name	nchar	(256) null,
instance_name	nchar	(256) null,
cntr_value		bigint	null
, second_value bigint null
)

--For databases both the primary and secondary, with send/receive counters reflecting the local replica
insert into @tempperfmon ([object_name],counter_name,instance_name,cntr_value)
select [object_name],counter_name,instance_name,cntr_value
from sys.dm_os_performance_counters
 where [object_name] like '%Availability Replica%' and instance_name <> '_Total' and
	(	counter_name = 'Bytes Received from Replica/sec' --From the availability replica. Pings and status updates will generate network traffic even on databases with no user updates.
	or	counter_name = 'Bytes Sent to Replica/sec' --Sent to the remote replica. On primary, sent to the secondary replica. On secondary, sent to the primary replica.
	or	counter_name = 'Bytes Sent to Transport/sec' --Sent over the network to the remote replica. On primary, sent to the secondary replica. On secondary, sent to the primary replica.
	or	counter_name = 'Flow Control Time (ms/sec)' --Time in milliseconds that log stream messages waited for send flow control, in the last second.   
	or	counter_name = 'Receives from Replica/Sec'
	or	counter_name = 'Sends to Replica/Sec'

	)
	
insert into @tempperfmon ([object_name],counter_name,instance_name,cntr_value)
--Only valid for databases in the secondary replica role
select [object_name],counter_name,instance_name,cntr_value
from sys.dm_os_performance_counters
 where [object_name] like '%database replica%' and instance_name <> '_Total' and
	(	counter_name = 'File Bytes Received/sec' --FILESTREAM data only
	or	counter_name = 'Log Bytes Received/sec' --Amount of log records received by the secondary replica for the database in the last second.'
	or	counter_name = 'Log remaining for undo' --The amount of log in kilobytes remaining to complete the undo phase.
	)

WAITFOR DELAY '00:00:05';  --5s


--For databases both the primary and secondary, with send/receive counters reflecting the local replica
insert into @tempperfmon ([object_name],counter_name,instance_name,second_value)
select [object_name],counter_name,instance_name,cntr_value
from sys.dm_os_performance_counters
 where [object_name] like '%Availability Replica%' and instance_name <> '_Total' and
	(	counter_name = 'Bytes Received from Replica/sec' --From the availability replica. Pings and status updates will generate network traffic even on databases with no user updates.
	or	counter_name = 'Bytes Sent to Replica/sec' --Sent to the remote replica. On primary, sent to the secondary replica. On secondary, sent to the primary replica.
	or	counter_name = 'Bytes Sent to Transport/sec' --Sent over the network to the remote replica. On primary, sent to the secondary replica. On secondary, sent to the primary replica.
	or	counter_name = 'Flow Control Time (ms/sec)' --Time in milliseconds that log stream messages waited for send flow control, in the last second.   
	or	counter_name = 'Receives from Replica/Sec'
	or	counter_name = 'Sends to Replica/Sec'

	)
	
insert into @tempperfmon ([object_name],counter_name,instance_name,second_value)
--Only valid for databases in the secondary replica role
select [object_name],counter_name,instance_name,cntr_value
from sys.dm_os_performance_counters
 where [object_name] like '%database replica%' and instance_name <> '_Total' and
	(	counter_name = 'File Bytes Received/sec' --FILESTREAM data only
	or	counter_name = 'Log Bytes Received/sec' --Amount of log records received by the secondary replica for the database in the last second.'
	or	counter_name = 'Log remaining for undo' --The amount of log in kilobytes remaining to complete the undo phase.
	)

select 
[object_name],counter_name,instance_name
, Observation = (max(second_value) - max(cntr_value)) /5.
from @tempperfmon
group by [object_name],counter_name,instance_name

/*
--For databases both the primary and secondary, with send/receive counters reflecting the local replica
select object_name,counter_name,instance_name,cntr_value
from sys.dm_os_performance_counters
 where object_name like '%Availability Replica%' and instance_name <> '_Total' and
	(	counter_name = 'Bytes Received from Replica/sec' --From the availability replica. Pings and status updates will generate network traffic even on databases with no user updates.
	or	counter_name = 'Bytes Sent to Replica/sec' --Sent to the remote replica. On primary, sent to the secondary replica. On secondary, sent to the primary replica.
	or	counter_name = 'Bytes Sent to Transport/sec' --Sent over the network to the remote replica. On primary, sent to the secondary replica. On secondary, sent to the primary replica.
	or	counter_name = 'Flow Control Time (ms/sec)' --Time in milliseconds that log stream messages waited for send flow control, in the last second.   
	or	counter_name = 'Receives from Replica/Sec'
	or	counter_name = 'Sends to Replica/Sec'

	)
	

--Only valid for databases in the secondary replica role
select object_name,counter_name,instance_name,cntr_value
from sys.dm_os_performance_counters
 where object_name like '%database replica%' and instance_name <> '_Total' and
	(	counter_name = 'File Bytes Received/sec' --FILESTREAM data only
	or	counter_name = 'Log Bytes Received/sec' --Amount of log records received by the secondary replica for the database in the last second.'
	or	counter_name = 'Log remaining for undo' --The amount of log in kilobytes remaining to complete the undo phase.
	)
*/

In [ ]:
--AG Replica Login Compare
PRINT('AG Replica Login Compare')
select * from sys.sql_logins